In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
f = 50
T = 1 / f
π = np.pi
ω = 2 * π * f

M_clark = np.sqrt(2/3) * np.array([
    [1, -1/2, -1/2],
    [0, np.sqrt(3)/2, -np.sqrt(3)/2],
    [1/np.sqrt(2), 1/np.sqrt(2), 1/np.sqrt(2)]
])
M_park = lambda θ: np.array([
    [np.cos(θ), np.sin(θ), 0],
    [-np.sin(θ), np.cos(θ), 0],
    [0, 0, 1]
])

In [ ]:
def dq0_tranform(ω, time, a, b, c):
    d = np.zeros_like(time)
    q = np.zeros_like(time)
    zero = np.zeros_like(time)
    for i, (t, x, y, z) in enumerate(zip(time, a, b, c)):
        θ = ω * t
        d[i], q[i], zero[i] = M_park(θ) @ M_clark @ np.array([x, y, z])
    return d, q, zero

In [ ]:
t = np.linspace(0, 2*T, 100)
ϕv = π / 10  # initial phase of the voltage
V = 10
va = V * np.cos(ω*t + ϕv)
vb = V * np.cos(ω*t + ϕv - 2 / 3 * π)
vc = V * np.cos(ω*t + ϕv + 2 / 3 * π)
vd, vq, v0 = dq0_tranform(ω, t, va, vb, vc)

In [ ]:
VA = V * np.exp(1j * (ϕv))
VB = V * np.exp(1j * (ϕv - 2 / 3 * π))
VC = V * np.exp(1j * (ϕv + 2 / 3 * π))

In [ ]:
R = 100   # [Ω]
L = 0.2   # [H]
Z = R + 1j * ω * L
abs_Z = np.abs(Z)
ϕ = np.atan2(Z.imag, Z.real)
print('Z = {} Ω'.format(Z))
print('ϕ = {:g} deg'.format(np.rad2deg(ϕ)))
print('cos(ϕ) = {:g}'.format(np.cos(ϕ)))

In [ ]:
correct_power_factor = True
COSPHI = 0.98
if correct_power_factor and np.cos(ϕ) < COSPHI:
    P_one_phase = 1 / 2 * V**2 / abs_Z * np.cos(ϕ)
    ϕr = np.acos(COSPHI)
    C = 2 * P_one_phase * (np.tan(ϕ) - np.tan(ϕr)) / (ω * V**2)
    Z = (Z / (1j*ω*C)) / (Z + 1/(1j*ω*C))
    abs_Z = np.abs(Z)
    ϕ = np.atan2(Z.imag, Z.real)
    print('C = {:g} μF'.format(C*1e6))
    print('Z = {:g} Ω'.format(Z))
    print('ϕ = {:g} deg'.format(np.rad2deg(ϕ)))
    print('cos(ϕ) = {:g}'.format(np.cos(ϕ)))

In [ ]:
IA = VA / Z
IB = VB / Z
IC = VC / Z

In [ ]:
ia = (IA * np.exp(1j * ω * t)).real
ib = (IB * np.exp(1j * ω * t)).real
ic = (IC * np.exp(1j * ω * t)).real
id, iq, i0 = dq0_tranform(ω, t, ia, ib, ic)
print('I = {:g} A'.format(ia.max()))

In [ ]:
P = 3 / 2 * V**2 / abs_Z * np.cos(ϕ)
Q = 3 / 2 * V**2 / abs_Z * np.sin(ϕ)
print(f'P = {P:g} W')
print(f'Q = {Q:g} VAR')

In [ ]:
assert np.max(np.abs((vd - vd[0]))) < 1e-12
assert abs(vd[0] * id[0] + vq[0] * iq[0] - P) < 1e-12
assert abs(vq[0] * id[0] - vd[0] * iq[0] - Q) < 1e-12

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(5,3), sharex=True)
ax[0].plot(t/T, va, 'r', lw=1, label='va')
ax[0].plot(t/T, vb, 'g', lw=1, label='vb')
ax[0].plot(t/T, vc, 'b', lw=1, label='vc')
ax[1].plot(t/T, vd, 'k', lw=1, label='vd')
ax[1].plot(t/T, vq, 'm', lw=1, label='vq')
ax[1].plot(t/T, v0, 'c--', lw=1, label='v0')
ax[1].set_xlabel('Time [T]')
ax[0].legend(loc='best', fontsize=8)
ax[1].legend(loc='best', fontsize=8)
sns.despine()
fig.tight_layout()

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(5,3), sharex=True)
ax[0].plot(t/T, ia, 'r', lw=1, label='ia')
ax[0].plot(t/T, ib, 'g', lw=1, label='ib')
ax[0].plot(t/T, ic, 'b', lw=1, label='ic')
ax[1].plot(t/T, id, 'k', lw=1, label='id')
ax[1].plot(t/T, iq, 'm', lw=1, label='iq')
ax[1].plot(t/T, i0, 'c--', lw=1, label='i0')
ax[1].set_xlabel('Time [T]')
ax[0].legend(loc='best', fontsize=8)
ax[1].legend(loc='best', fontsize=8)
sns.despine()
fig.tight_layout()